In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV

In [2]:
train_df = pd.read_csv("D:\\SI Rourkela\\UNSW_NB15\\UNSW_NB15_training-set.csv")
test_df = pd.read_csv("D:\\SI Rourkela\\UNSW_NB15\\UNSW_NB15_testing-set.csv")

In [3]:
train_df

,id,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,...,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
0,1,0.000011,udp,-,INT,2,0,496,0,90909.090200,...,1,2,0,0,0,1,2,0,Normal,0
1,2,0.000008,udp,-,INT,2,0,1762,0,125000.000300,...,1,2,0,0,0,1,2,0,Normal,0
2,3,0.000005,udp,-,INT,2,0,1068,0,200000.005100,...,1,3,0,0,0,1,3,0,Normal,0
3,4,0.000006,udp,-,INT,2,0,900,0,166666.660800,...,1,3,0,0,0,2,3,0,Normal,0
4,5,0.000010,udp,-,INT,2,0,2126,0,100000.002500,...,1,3,0,0,0,2,3,0,Normal,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82327,82328,0.000005,udp,-,INT,2,0,104,0,200000.005100,...,1,2,0,0,0,2,1,0,Normal,0
82328,82329,1.106101,tcp,-,FIN,20,8,18062,354,24.410067,...,1,1,0,0,0,3,2,0,Normal,0
82329,82330,0.000000,arp,-,INT,1,0,46,0,0.000000,...,1,1,0,0,0,1,1,1,Normal,0
82330,82331,0.000000,arp,-,INT,1,0,46,0,0.000000,...,1,1,0,0,0,1,1,1,Normal,0


In [4]:
test_df

,id,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,...,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
0,1,0.121478,tcp,-,FIN,6,4,258,172,74.087490,...,1,1,0,0,0,1,1,0,Normal,0
1,2,0.649902,tcp,-,FIN,14,38,734,42014,78.473372,...,1,2,0,0,0,1,6,0,Normal,0
2,3,1.623129,tcp,-,FIN,8,16,364,13186,14.170161,...,1,3,0,0,0,2,6,0,Normal,0
3,4,1.681642,tcp,ftp,FIN,12,12,628,770,13.677108,...,1,3,1,1,0,2,1,0,Normal,0
4,5,0.449454,tcp,-,FIN,10,6,534,268,33.373826,...,1,40,0,0,0,2,39,0,Normal,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175336,175337,0.000009,udp,dns,INT,2,0,114,0,111111.107200,...,13,24,0,0,0,24,24,0,Generic,1
175337,175338,0.505762,tcp,-,FIN,10,8,620,354,33.612649,...,1,2,0,0,0,1,1,0,Shellcode,1
175338,175339,0.000009,udp,dns,INT,2,0,114,0,111111.107200,...,3,13,0,0,0,3,12,0,Generic,1
175339,175340,0.000009,udp,dns,INT,2,0,114,0,111111.107200,...,14,30,0,0,0,30,30,0,Generic,1


In [5]:
for df in [train_df,test_df]:
    df.drop(columns=['id'], inplace=True)

In [6]:
train_df

,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,sttl,...,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
0,0.000011,udp,-,INT,2,0,496,0,90909.090200,254,...,1,2,0,0,0,1,2,0,Normal,0
1,0.000008,udp,-,INT,2,0,1762,0,125000.000300,254,...,1,2,0,0,0,1,2,0,Normal,0
2,0.000005,udp,-,INT,2,0,1068,0,200000.005100,254,...,1,3,0,0,0,1,3,0,Normal,0
3,0.000006,udp,-,INT,2,0,900,0,166666.660800,254,...,1,3,0,0,0,2,3,0,Normal,0
4,0.000010,udp,-,INT,2,0,2126,0,100000.002500,254,...,1,3,0,0,0,2,3,0,Normal,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82327,0.000005,udp,-,INT,2,0,104,0,200000.005100,254,...,1,2,0,0,0,2,1,0,Normal,0
82328,1.106101,tcp,-,FIN,20,8,18062,354,24.410067,254,...,1,1,0,0,0,3,2,0,Normal,0
82329,0.000000,arp,-,INT,1,0,46,0,0.000000,0,...,1,1,0,0,0,1,1,1,Normal,0
82330,0.000000,arp,-,INT,1,0,46,0,0.000000,0,...,1,1,0,0,0,1,1,1,Normal,0


In [7]:
test_df

,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,sttl,...,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
0,0.121478,tcp,-,FIN,6,4,258,172,74.087490,252,...,1,1,0,0,0,1,1,0,Normal,0
1,0.649902,tcp,-,FIN,14,38,734,42014,78.473372,62,...,1,2,0,0,0,1,6,0,Normal,0
2,1.623129,tcp,-,FIN,8,16,364,13186,14.170161,62,...,1,3,0,0,0,2,6,0,Normal,0
3,1.681642,tcp,ftp,FIN,12,12,628,770,13.677108,62,...,1,3,1,1,0,2,1,0,Normal,0
4,0.449454,tcp,-,FIN,10,6,534,268,33.373826,254,...,1,40,0,0,0,2,39,0,Normal,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175336,0.000009,udp,dns,INT,2,0,114,0,111111.107200,254,...,13,24,0,0,0,24,24,0,Generic,1
175337,0.505762,tcp,-,FIN,10,8,620,354,33.612649,254,...,1,2,0,0,0,1,1,0,Shellcode,1
175338,0.000009,udp,dns,INT,2,0,114,0,111111.107200,254,...,3,13,0,0,0,3,12,0,Generic,1
175339,0.000009,udp,dns,INT,2,0,114,0,111111.107200,254,...,14,30,0,0,0,30,30,0,Generic,1


In [8]:
cat_cols = ['proto', 'state', 'service']
for col in cat_cols:
    freq = train_df[col].value_counts() / len(train_df)
    train_df[col] = train_df[col].map(freq)
    test_df[col] = test_df[col].map(freq).fillna(0)

In [9]:
train_df

,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,sttl,...,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
0,0.000011,0.357309,0.572718,0.414942,2,0,496,0,90909.090200,254,...,1,2,0,0,0,1,2,0,Normal,0
1,0.000008,0.357309,0.572718,0.414942,2,0,1762,0,125000.000300,254,...,1,2,0,0,0,1,2,0,Normal,0
2,0.000005,0.357309,0.572718,0.414942,2,0,1068,0,200000.005100,254,...,1,3,0,0,0,1,3,0,Normal,0
3,0.000006,0.357309,0.572718,0.414942,2,0,900,0,166666.660800,254,...,1,3,0,0,0,2,3,0,Normal,0
4,0.000010,0.357309,0.572718,0.414942,2,0,2126,0,100000.002500,254,...,1,3,0,0,0,2,3,0,Normal,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82327,0.000005,0.357309,0.572718,0.414942,2,0,104,0,200000.005100,254,...,1,2,0,0,0,2,1,0,Normal,0
82328,1.106101,0.523430,0.572718,0.477809,20,8,18062,354,24.410067,254,...,1,1,0,0,0,3,2,0,Normal,0
82329,0.000000,0.011988,0.572718,0.414942,1,0,46,0,0.000000,0,...,1,1,0,0,0,1,1,1,Normal,0
82330,0.000000,0.011988,0.572718,0.414942,1,0,46,0,0.000000,0,...,1,1,0,0,0,1,1,1,Normal,0


In [10]:
test_df

,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,sttl,...,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
0,0.121478,0.523430,0.572718,0.477809,6,4,258,172,74.087490,252,...,1,1,0,0,0,1,1,0,Normal,0
1,0.649902,0.523430,0.572718,0.477809,14,38,734,42014,78.473372,62,...,1,2,0,0,0,1,6,0,Normal,0
2,1.623129,0.523430,0.572718,0.477809,8,16,364,13186,14.170161,62,...,1,3,0,0,0,2,6,0,Normal,0
3,1.681642,0.523430,0.018851,0.477809,12,12,628,770,13.677108,62,...,1,3,1,1,0,2,1,0,Normal,0
4,0.449454,0.523430,0.572718,0.477809,10,6,534,268,33.373826,254,...,1,40,0,0,0,2,39,0,Normal,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175336,0.000009,0.357309,0.259522,0.414942,2,0,114,0,111111.107200,254,...,13,24,0,0,0,24,24,0,Generic,1
175337,0.505762,0.523430,0.572718,0.477809,10,8,620,354,33.612649,254,...,1,2,0,0,0,1,1,0,Shellcode,1
175338,0.000009,0.357309,0.259522,0.414942,2,0,114,0,111111.107200,254,...,3,13,0,0,0,3,12,0,Generic,1
175339,0.000009,0.357309,0.259522,0.414942,2,0,114,0,111111.107200,254,...,14,30,0,0,0,30,30,0,Generic,1


In [11]:
# X_train = train_df.drop(columns=['label', 'attack_cat'])
# y_train = train_df['label']
# X_test = test_df.drop(columns=['label', 'attack_cat'])
# y_test = test_df['label']

In [12]:
# import numpy as np

# # Save your test data
# np.save('X_test.npy', X_test)
# np.save('y_test.npy', y_test)

In [13]:
# from xgboost import XGBClassifier

# model = XGBClassifier(
#     n_estimators=200,          # Number of boosting rounds/trees
#     max_depth=6,               # Max depth of trees
#     learning_rate=0.1,         # Shrinkage rate
#     eval_metric='logloss',     # Evaluation metric for binary classification
#     scale_pos_weight=2,        # Handle class imbalance
#     random_state=42            # For reproducibility
# )
# model.fit(X_train, y_train)

In [14]:
# y_pred = model.predict(X_test)

# print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
# print("Classification Report:\n", classification_report(y_test, y_pred))
# print("Accuracy:", accuracy_score(y_test, y_pred))

In [15]:
# param_dist = {
#     'n_estimators': [100, 200, 300],
#     'max_depth': [4, 6, 8, 10],
#     'learning_rate': [0.01, 0.05, 0.1],
#     'subsample': [0.8, 1.0],
#     'colsample_bytree': [0.7, 0.8, 1.0],
#     'scale_pos_weight': [2, 3, 5],  # Key param for class imbalance
# }

# model = XGBClassifier(eval_metric='logloss', random_state=42)
# search = RandomizedSearchCV(model, param_distributions=param_dist, scoring='recall', n_iter=10, cv=3, verbose=1)
# search.fit(X_train, y_train)

# best_model = search.best_estimator_
# y_pred = best_model.predict(X_test)
# y_proba = best_model.predict_proba(X_test)[:, 1]
# y_pred_thresh = (y_proba >= 0.4).astype(int)  # Lower than 0.5 to reduce FN

# from sklearn.metrics import confusion_matrix, classification_report
# print(confusion_matrix(y_test, y_pred_thresh))
# print(classification_report(y_test, y_pred_thresh))

In [16]:
# (y_train == 0).sum()

In [17]:
# (y_train == 1).sum()

In [18]:
# from sklearn.metrics import precision_recall_curve
# import matplotlib.pyplot as plt

# y_proba = best_model.predict_proba(X_test)[:, 1]

# precision, recall, thresholds = precision_recall_curve(y_test, y_proba)

# plt.plot(thresholds, precision[:-1], label='Precision')
# plt.plot(thresholds, recall[:-1], label='Recall')
# plt.xlabel('Threshold')
# plt.legend()
# plt.show()

In [19]:
# t = 0.36
# y_pred_thresh = (y_proba >= t).astype(int)
# print(f"\nThreshold: {t}")
# print(confusion_matrix(y_test, y_pred_thresh))
# print(classification_report(y_test, y_pred_thresh))

In [20]:
# from xgboost import XGBClassifier
# from sklearn.model_selection import RandomizedSearchCV
# from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score

# # Hyperparameter grid
# param_dist = {
#     'n_estimators': [100, 200, 300],
#     'max_depth': [4, 6, 8, 10],
#     'learning_rate': [0.01, 0.05, 0.1],
#     'subsample': [0.8, 1.0],
#     'colsample_bytree': [0.7, 0.8, 1.0],
#     'scale_pos_weight': [2, 3, 5],
# }

# # Model + RandomizedSearchCV
# model = XGBClassifier(eval_metric='logloss', random_state=42)
# search = RandomizedSearchCV(model, param_distributions=param_dist,
#                             scoring='recall', n_iter=10, cv=3,
#                             verbose=1, n_jobs=-1)
# search.fit(X_train, y_train)

# # Best model and probability predictions
# best_model = search.best_estimator_
# y_proba = best_model.predict_proba(X_test)[:, 1]

# # Apply optimal threshold
# threshold = 0.36
# y_pred_thresh = (y_proba >= threshold).astype(int)

# # Final evaluation
# print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_thresh))
# print("Classification Report:\n", classification_report(y_test, y_pred_thresh))
# print("ROC AUC Score:", roc_auc_score(y_test, y_proba))

In [21]:
# y_proba = best_model.predict_proba(X_test)[:, 1]
# optimal_threshold = 0.24  # or whatever gave you best balance
# y_pred_thresh = (y_proba >= optimal_threshold).astype(int)

# # Final Evaluation
# print(confusion_matrix(y_test, y_pred_thresh))
# print(classification_report(y_test, y_pred_thresh))

In [22]:
# from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score

# # Predict probabilities
# y_proba = best_model.predict_proba(X_test)[:, 1]

# # ✅ Use correct threshold (not 0 or 0.01)
# optimal_threshold = 0.36

# # Predict based on threshold
# y_pred_thresh = (y_proba >= optimal_threshold).astype(int)

# # Evaluation
# print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_thresh))
# print("\nClassification Report:\n", classification_report(y_test, y_pred_thresh))

# # Optional: ROC AUC Score
# print("\nROC AUC Score:", roc_auc_score(y_test, y_proba))

In [23]:
# import joblib
# param_dist = {
#     'n_estimators': [100, 200, 300],
#     'max_depth': [4, 6, 8, 10],
#     'learning_rate': [0.01, 0.05, 0.1],
#     'subsample': [0.8, 1.0],
#     'colsample_bytree': [0.7, 0.8, 1.0],
#     'scale_pos_weight': [2, 3, 5],
# }

# # Randomized search with cross-validation
# model = XGBClassifier(eval_metric='logloss', random_state=42)
# search = RandomizedSearchCV(model, param_distributions=param_dist,
#                             scoring='recall', n_iter=10, cv=3, verbose=1, n_jobs=-1)
# search.fit(X_train, y_train)
# best_model = search.best_estimator_

# # Predict probabilities
# y_proba = best_model.predict_proba(X_test)[:, 1]

# # Optimal threshold based on precision-recall analysis
# optimal_threshold = 0.30

# # Save the model, threshold, and test data
# joblib.dump(best_model, "xgboost_final_model.pkl")
# with open("optimal_threshold.txt", "w") as f:
#     f.write(str(optimal_threshold))

# print("Training complete. Model and threshold saved.")

# # ======================
# # File: evaluate_model.py
# # ======================

# import numpy as np
# import joblib
# from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score

# # Load model and threshold
# model = joblib.load("xgboost_final_model.pkl")
# with open("optimal_threshold.txt", "r") as f:
#     threshold = float(f.read())

# # Load test data
# X_test = np.load("X_test.npy")
# y_test = np.load("y_test.npy")

# # Predict and evaluate
# y_proba = model.predict_proba(X_test)[:, 1]
# y_pred = (y_proba >= threshold).astype(int)

# print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
# print("\nClassification Report:\n", classification_report(y_test, y_pred))
# print("\nROC AUC Score:", roc_auc_score(y_test, y_proba))

In [24]:
import numpy as np
import pandas as pd
import joblib
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score
import random
import os

# =============== Set Random Seeds (For Full Reproducibility) ===============
SEED = 42
np.random.seed(SEED)
random.seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)

# Optional: Limit number of threads for consistent results
import xgboost
xgboost.set_config(verbosity=0)  # silence warnings

# ================= Load Your Train/Test Data =================
# Assuming train_df and test_df are already defined
X_train = train_df.drop(columns=['label', 'attack_cat'])
y_train = train_df['label']
X_test = test_df.drop(columns=['label', 'attack_cat'])
y_test = test_df['label']

# ================= Save X_test & y_test =================
np.save('X_test.npy', X_test.to_numpy())
np.save('y_test.npy', y_test.to_numpy())

# ================ Hyperparameter Tuning ====================
param_dist = {
    'n_estimators': [100, 200, 300],
    'max_depth': [4, 6, 8, 10],
    'learning_rate': [0.01, 0.05, 0.1],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.7, 0.8, 1.0],
    'scale_pos_weight': [2, 3, 5],
}

model = XGBClassifier(eval_metric='logloss', random_state=SEED, n_jobs=1)  # <- limit threads

search = RandomizedSearchCV(model,
                            param_distributions=param_dist,
                            scoring='recall',
                            n_iter=10,
                            cv=3,
                            verbose=1,
                            n_jobs=1,
                            random_state=SEED)
search.fit(X_train, y_train)

best_model = search.best_estimator_

# ================ Apply Threshold ====================
y_proba = best_model.predict_proba(X_test)[:, 1]
optimal_threshold = 0.30
y_pred_thresh = (y_proba >= optimal_threshold).astype(int)

print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_thresh))
print("Classification Report:\n", classification_report(y_test, y_pred_thresh))
print("ROC AUC Score:", roc_auc_score(y_test, y_proba))

# ================ Save Everything ====================
joblib.dump(best_model, "xgboost_binary_model.pkl")
with open("optimal_threshold.txt", "w") as f:
    f.write(str(optimal_threshold))

print("✅ Model, threshold, and test data saved successfully.")

Fitting 3 folds for each of 10 candidates, totalling 30 fits
Confusion Matrix:
 [[ 31076  24924]
 [     0 119341]]
Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.55      0.71     56000
           1       0.83      1.00      0.91    119341

    accuracy                           0.86    175341
   macro avg       0.91      0.78      0.81    175341
weighted avg       0.88      0.86      0.84    175341

ROC AUC Score: 0.9825192124278928
✅ Model, threshold, and test data saved successfully.
